In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import tensorflow_addons as tfa

c:\Users\sagoi\OneDrive\Desktop\Grammarly\venv\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\sagoi\OneDrive\Desktop\Grammarly\venv\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do no

In [2]:
df = pd.read_csv("./data/data1.csv",nrows=5)

incorrect_sentences = df['input']
correct_sentences = df['target']

In [3]:
incorrect_sentences

0    Bitcoin is for $7,094 this morning, which Coin...
1    The effect of widespread dud targets two face ...
2    tax on sales of stores for non residents are s...
3    Much many brands and sellers still in the market.
4    this is is the latest Maintenance release of S...
Name: input, dtype: object

In [4]:
correct_sentences

0    Bitcoin goes for $7,094 this morning, accordin...
1    1. The effect of "widespread dud" targets two ...
2    Capital Gains tax on the sale of properties fo...
3         Many brands and sellers still in the market.
4    This is is the latest maintenance release of S...
Name: target, dtype: object

In [9]:
input_characters = set(' '.join(incorrect_sentences + correct_sentences))
target_characters = input_characters

In [6]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = input_token_index

In [7]:
max_encoder_seq_length = max([len(txt) for txt in incorrect_sentences])
max_decoder_seq_length = max([len(txt) for txt in correct_sentences])

In [8]:
encoder_input_data = np.zeros((len(incorrect_sentences), max_encoder_seq_length, len(input_characters)), dtype='float32')
decoder_input_data = np.zeros((len(incorrect_sentences), max_decoder_seq_length, len(target_characters)), dtype='float32')
decoder_target_data = np.zeros((len(incorrect_sentences), max_decoder_seq_length, len(target_characters)), dtype='float32')

In [ ]:
for i, (input_text, target_text) in enumerate(zip(incorrect_sentences, correct_sentences)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

In [25]:
encoder_input_data

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

    